In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.cm as cm
import numpy as np
import matplotlib as mpl
from scipy.signal import detrend

In [3]:
def detrend_da(data):
    data.data = detrend(data.data, axis=0)
    return data

In [4]:
def read_data(path):
    dsets = []
    for p in path:
        dsets.append(xr.open_dataset(p))
    return xr.merge(dsets)

noresm = read_data(snakemake.input.noresm).isel(time=slice(5,None))

mpi = read_data(snakemake.input.mpiEsm).isel(time=slice(5,None))

ecEarth = read_data(snakemake.input.ecEarth).isel(time=slice(5,None))

if snakemake.params.get('detrend',None):
    odvarEC = detrend_da(ecEarth['od550aer'].isel(time=slice(-30,None))).std(dim='time')
    odvarNO = detrend_da(noresm['od550aer'].isel(time=slice(-30,None))).std(dim='time')
    odvarMPI = detrend_da(mpi['od550aer'].isel(time=slice(-30,None))).std(dim='time')

    odvarEC_PI = detrend_da(ecEarth['od550aer'].dropna(dim='time').isel(time=slice(None,30))).std(dim='time')
    odvarNO_PI = detrend_da(noresm['od550aer'].isel(time=slice(None,30))).std(dim='time')
    odvarMPI_PI = detrend_da(mpi['od550aer'].isel(time=slice(None,30))).std(dim='time')
    main_t = 'Variability of Detrended antropogenic AOD histSST simulations'
else:
    odvarEC = ecEarth['od550aer'].isel(time=slice(-30,None)).std(dim='time')
    odvarNO = noresm['od550aer'].isel(time=slice(-30,None)).std(dim='time')
    odvarMPI = mpi['od550aer'].isel(time=slice(-30,None)).std(dim='time')

    odvarEC_PI = ecEarth['od550aer'].dropna(dim='time').isel(time=slice(None,30)).std(dim='time')
    odvarNO_PI = noresm['od550aer'].isel(time=slice(None,30)).std(dim='time')
    odvarMPI_PI = mpi['od550aer'].isel(time=slice(None,30)).std(dim='time')
    main_t = 'Variability of antropogenic AOD histSST simulations'

In [5]:
da_list = [odvarEC,odvarNO,odvarMPI]
titles = [ecEarth.source_id,noresm.source_id,mpi.source_id]
da_list_pi = [odvarEC_PI,odvarNO_PI,odvarMPI_PI]
fig ,ax = plt.subplots(nrows=3,ncols=3, subplot_kw={'projection':ccrs.Robinson()}, figsize=(15,9))
cmap = cm.get_cmap('YlOrRd',11)
cmap.set_under('k', alpha=0.0)


for a, da,t in zip(ax[:,0], da_list, titles):
    a.coastlines()
    da.plot(ax=a, transform=ccrs.PlateCarree(), cmap = cmap, vmin=0.01, vmax=0.3, extend='neither', add_colorbar=False)
    a.set_title(t)
for a, da,t in zip(ax[:,1], da_list_pi, titles):
    a.coastlines()
    da.plot(ax=a, transform=ccrs.PlateCarree(), cmap = cmap, vmin=0.01, vmax=0.3, extend='neither', add_colorbar=False)
    a.set_title(t)
cax = fig.add_axes([0.2,0.1,0.33,0.02])
cb = fig.colorbar(mappable=cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=0.01,vmax=0.3),cmap=cmap),
                  cax=cax,orientation='horizontal', format='%.2f')
cb.set_ticks(np.linspace(0.01,0.3,12))
cmap = cm.get_cmap('PRGn',11)

for a, da,da_pi,t in zip(ax[:,2], da_list,da_list_pi, titles):
    a.coastlines()
    da = da - da_pi
    da.plot(ax=a, transform=ccrs.PlateCarree(), cmap = cmap, vmin=-0.2, vmax=0.2, extend='neither', add_colorbar=False)
    a.set_title(t)
cax = fig.add_axes([0.7,0.1,0.17,0.02])
cb = fig.colorbar(mappable=cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=-0.2,vmax=0.2),cmap=cmap),
                      cax=cax,orientation='horizontal')

fig.suptitle(main_t, fontsize=16)
fig.text(0.198, 0.93, 'Last 30 years', va='center', fontsize=14, color='#c314cc')
fig.text(0.471, 0.93, 'First 30 years', va='center', fontsize=14, color='#b56010')
fig.text(0.7, 0.93, 'Last 30 years - First 30 years', va='center', fontsize=14, color='#0a9186')
plt.savefig(snakemake.output.aod_varibility_map, dpi=144, bbox_inches='tight')